In [1]:
import requests
import json
import time

# Define the LLM endpoint and model details
LLM_ENDPOINT = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3.2:3b"

# Seed list of initial professions
seed_professions = [
    "Nuclear Engineer",
    "Software Engineer",
    "Data Scientist",
    "Mechanical Engineer",
    "Electrical Engineer",
    "Aerospace Engineer",
    "Chemical Engineer",
    "Civil Engineer",
    "Biomedical Engineer",
    "Environmental Scientist"
]

# Master list to keep track of professions and their related jobs
master_professions = set(seed_professions)
related_professions_map = {}  # Dictionary to store the output JSON of each profession

# Define a function to send a request to the LLM
def query_llm(profession):
    prompt = f'Give me a list of related professions to "{profession}". Try to form a list of 5-10. Respond in JSON like: {{"profession": "{profession}", "related_professions": ["profession_1", "profession_2", ]}} Only respond with the JSON and nothing else. Be specific with the job title: instead of "producer", use "music producer".'
    
    # Send the request to the local LLM
    response = requests.post(
        LLM_ENDPOINT,
        data=json.dumps({
            "model": MODEL_NAME,
            "prompt": prompt,
            "stream": False
        }),
        headers={"Content-Type": "application/json"}
    )

    # Parse and return the JSON response
    if response.status_code == 200:
        response_json = response.json()
        return response_json.get("response", "{}")  # Get the "response" field if present
    else:
        print(f"Error: Received status code {response.status_code}")
        return None



In [1]:
import requests
import json
import time
from collections import deque

# Define the LLM endpoint and model details
LLM_ENDPOINT = "http://localhost:11434/api/chat"
MODEL_NAME = "llama3.2"

# Seed list of initial professions
seed_professions = [
    "Astronaut",
    "Fashion Designer",
    "Sommelier",
    "Cryptographer",
    "Zoologist",
    "Marine Biologist",
    "DJ",
    "Librarian",
    "Tattoo Artist",
    "Ballet Dancer"
]

# Master list to keep track of all queried professions
master_professions = set(seed_professions)
related_professions_map = {}  # Dictionary to store the output JSON of each profession

# Queue to track professions that need to be queried
query_queue = deque(seed_professions)

# Set a maximum limit for the number of queries
MAX_QUERIES = 25
current_query_count = 0

# Maximum retries for getting a valid response
MAX_RETRIES = 3


# Function to send a request to the LLM using message history for context
def query_llm_with_context(conversation_history):
    response = requests.post(
        LLM_ENDPOINT,
        data=json.dumps({
            "model": MODEL_NAME,
            "messages": conversation_history,
            "stream": False
        }),
        headers={"Content-Type": "application/json"}
    )

    # Parse and return the JSON response
    if response.status_code == 200:
        response_json = response.json()
        return response_json.get("response", "{}")  # Get the "response" field if present
    else:
        print(f"Error: Received status code {response.status_code}")
        return None


# Validate the response format and ensure it contains 'related_professions'
def validate_response(response_data, profession):
    try:
        parsed_data = json.loads(response_data)
        # Check if the necessary fields are present in the response
        if "profession" in parsed_data and "related_professions" in parsed_data:
            return True
    except json.JSONDecodeError:
        pass

    # If invalid, print the invalid response for review
    print(f"Invalid response for {profession}: {response_data}")
    return False


# Loop through the queue until we've reached the maximum query limit or run out of items in the queue
while query_queue and current_query_count < MAX_QUERIES:
    # Get the next profession to query
    profession = query_queue.popleft()
    
    print(f"Querying for profession: {profession} (Query {current_query_count+1}/{MAX_QUERIES})")

    # Build initial message for the context
    conversation_history = [
        {
            "role": "user",
            "content": f'Give me a list of related professions to "{profession}". Try to form a list of 5-10. Respond in JSON like: {{"profession": "{profession}", "related_professions": ["profession_1", "profession_2", ]}} Only respond with the JSON and nothing else. Be specific with the job title: instead of "producer", use "music producer".'
        }
    ]

    response = query_llm_with_context(conversation_history)
    retry_count = 0
    
    # Retry if the response is invalid or if JSON parsing fails
    while response and not validate_response(response, profession) and retry_count < MAX_RETRIES:
        retry_count += 1
        print(f"Retrying for {profession} (Attempt {retry_count}/{MAX_RETRIES})")
        
        # Add assistant's response and user's correction prompt to the conversation history
        conversation_history.append({
            "role": "assistant",
            "content": response  # Add the assistant's last response
        })
        conversation_history.append({
            "role": "user",
            "content": f'The response should include a "related_professions" key. Your response did not include it for {profession}. Please respond again in JSON format like: {{"profession": "{profession}", "related_professions": ["profession_1", "profession_2", ]}}.'
        })
        
        # Send the corrected request with updated conversation history
        response = query_llm_with_context(conversation_history)

    # If we have a valid response, process it
    if validate_response(response, profession):
        try:
            # Parse the JSON response and store it
            response_data = json.loads(response)
            related_professions_map[profession] = response_data

            # Add new related professions to the master list and queue if they are strings and not already queried
            for related_prof in response_data["related_professions"]:
                if isinstance(related_prof, str) and related_prof not in master_professions:
                    master_professions.add(related_prof)
                    query_queue.append(related_prof)

            # Increment the query count and pause
            current_query_count += 1
            time.sleep(1)
        except json.JSONDecodeError:
            print(f"Failed to parse JSON for profession: {profession}")
            continue

# Display the final output JSON and save it
print("Related Professions Map:")
print(json.dumps(related_professions_map, indent=4))

# Save the result as a JSON file
with open("related_professions_map.json", "w") as file:
    json.dump(related_professions_map, file, indent=4)

# Show master list of all professions discovered
print(f"Total unique professions found: {len(master_professions)}")
print(master_professions)


Querying for profession: Astronaut (Query 1/25)
Error: Received status code 404


TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [ ]:
import requests
import json

# Configuration
seed_professions = [
    "Astronaut", "Fashion Designer", "Sommelier", "Cryptographer", 
    "Zoologist", "Marine Biologist", "DJ", "Librarian", "Tattoo Artist", "Ballet Dancer"
]

# Local LLM API configuration
api_url = "http://localhost:11434/api/generate"
model_name = "llama3.2:3b"

# Track queried professions and responses
queried_professions = set()
all_professions = set(seed_professions)
json_responses = []
query_limit = 1000  # Limit the number of queries

# Define a function to send a request to the local LLM
def query_llm(profession):
    prompt = (
        f'Give me a list of related professions to "{profession}". '
        f'Try to form a list of 5-10. Respond in JSON like: '
        f'{{"profession": "{profession}", "related_professions": ["profession_1", "profession_2", ]}}. '
        "Only respond with the JSON and nothing else. Be specific with the job title: instead of 'producer', use 'music producer'."
    )
    
    data = {
        "model": model_name,
        "prompt": prompt,
        "stream": False
    }
    
    try:
        response = requests.post(api_url, json=data)
        response_data = response.json()
        if "response" in response_data:
            return response_data["response"]
        else:
            print(f"Error: No 'response' key in LLM response for profession: {profession}")
            return None
    except json.JSONDecodeError:
        # Handle JSON decode error by issuing a correction prompt
        correction_prompt = (
            f'The response should include a "related_professions" key with an array of strings for each related profession. '
            f'Your response did not include it for {profession}. Please respond again in JSON format like: '
            f'{{"profession": "{profession}", "related_professions": ["profession_1", "profession_2", ]}}.'
        )
        data["prompt"] = correction_prompt
        response = requests.post(api_url, json=data)
        response_data = response.json()
        return response_data.get("response", None)
    except Exception as e:
        print(f"Error querying profession {profession}: {e}")
        return None

# Validate the response format and correct it if necessary
def validate_and_correct_json(profession, response_text):
    try:
        # Attempt to parse the JSON response
        parsed_response = json.loads(response_text)

        # Check if the "related_professions" key is a list of strings
        if isinstance(parsed_response.get("related_professions", []), list):
            # Check if all items in the list are strings
            if all(isinstance(item, str) for item in parsed_response["related_professions"]):
                return parsed_response
            else:
                print(f"Invalid response format for profession {profession}. Retrying correction...")
        else:
            print(f"Response for {profession} did not include a valid 'related_professions' array. Retrying correction...")

    except json.JSONDecodeError:
        print(f"Failed to decode JSON for profession {profession}: {response_text}")
    
    # If the response was invalid, send a correction prompt
    correction_prompt = (
        f'The response should include a "related_professions" key with an array of strings for each related profession. '
        f'Your response did not include it for {profession}. Please respond again in JSON format like: '
        f'{{"profession": "{profession}", "related_professions": ["profession_1", "profession_2", ]}}.'
    )
    
    # Re-query with the correction prompt
    corrected_response = query_llm_with_correction(profession, correction_prompt)
    if corrected_response:
        try:
            corrected_parsed_response = json.loads(corrected_response)
            return corrected_parsed_response
        except json.JSONDecodeError:
            print(f"Failed to decode corrected JSON for profession {profession}: {corrected_response}")
            return None
    return None

# Function to send a correction prompt to the LLM
def query_llm_with_correction(profession, correction_prompt):
    data = {
        "model": model_name,
        "prompt": correction_prompt,
        "stream": False
    }
    try:
        response = requests.post(api_url, json=data)
        response_data = response.json()
        if "response" in response_data:
            return response_data["response"]
        else:
            print(f"Error: No 'response' key in corrected LLM response for profession: {profession}")
            return None
    except Exception as e:
        print(f"Error during correction for profession {profession}: {e}")
        return None

# Main iterative querying loop
def iterative_profession_query(seed_professions, query_limit):
    global queried_professions, all_professions, json_responses

    professions_to_query = list(seed_professions)
    while professions_to_query and len(queried_professions) < query_limit:
        profession = professions_to_query.pop(0)
        
        # Skip if we've already queried this profession
        if profession in queried_professions:
            continue
        
        # Query the LLM and parse the response
        print(f"Querying profession: {profession}")
        json_response = query_llm(profession)
        
        if json_response:
            parsed_response = validate_and_correct_json(profession, json_response)
            if parsed_response:
                json_responses.append(parsed_response)
                queried_professions.add(profession)
                
                # Extract new professions and add them to the list
                related_professions = parsed_response.get("related_professions", [])
                new_professions = set(related_professions) - all_professions
                professions_to_query.extend(new_professions)
                all_professions.update(new_professions)
                
                print(f"Found related professions for {profession}: {related_professions}")
            else:
                print(f"Failed to get valid response for profession {profession}")
        else:
            print(f"No response or invalid response for profession: {profession}")

# Run the iterative querying process
iterative_profession_query(seed_professions, query_limit)

# Save the results
with open("related_professions.json", "w") as f:
    json.dump(json_responses, f, indent=4)

print("Saved related professions to 'related_professions.json'.")


Querying profession: Astronaut
Found related professions for Astronaut: ['Cosmonaut', 'Astrophysicist', 'Planetary Scientist', 'Rocket Engineer', 'Space Biologist', 'Flight Director', 'Test Pilot', 'Air Traffic Controller', 'Scientific Researcher', 'Geophysicist']
Querying profession: Fashion Designer
Found related professions for Fashion Designer: ['Costume Designer', 'Pattern Maker', 'Fashion Illustrator', 'Textile Designer', 'Clothing Buyer', 'Accessories Designer', 'Haute Couture Designer', 'Wardrobe Stylist', 'Fashion Editor', 'Fashion Writer']
Querying profession: Sommelier
Found related professions for Sommelier: ['Wine Consultant', 'Food and Wine Pairing Specialist', 'Catering Manager', 'Restaurant Manager', 'Bar Manager', 'Mixologist', 'Beverage Director', 'Sommelier Manager', 'Wine Educator', 'Tasting Room Manager']
Querying profession: Cryptographer
Found related professions for Cryptographer: ['Computer Security Specialist', 'Cybersecurity Analyst', 'Information Security En